In [104]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
from __future__ import print_function

import datetime
import pandas as pd
import numpy as np
from dateutil import parser as dtparser

from utils.datafetch import *
from utils.vectorized_funs import *
from utils.datapipe import *
from utils.kerasutil import *
from utils import scenarioa
from FintechCapstone import FinCapstone

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [106]:
%matplotlib inline

plt.style.use('seaborn-whitegrid')
date_from = '2009-01-01'
date_to = '2017-03-01'
timespan = {
    "short_term": [1, 5]
    ,"medium_term": [40]
    ,"long_term": [90]
}

trial = FinCapstone(ticker_list=["GLW","HIMX","GLUU"], ticker_list_samplesize=4, timespan=timespan, date_from=date_from)
#trial = FinCapstone(ticker_list=["ZHNE","BSFT","AERO","MATR","ITMSF","FIT","TIPT","OLBK","QADA","ATTU","LGF","STX","TPCS","GSAT","ZNGA","GLW","HIMX","YHOO", "NWSA","JAKK","GLUU","AMZN", "AAPL", "EBAY", "GOOG", "DIS", "NFLX", "EA", "TWTR", "FB", "TTWO", "PXLW", "UBI"], ticker_list_samplesize=4, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)
#trial = FinCapstone(ticker_list_samplesize=200, timespan=timespan, timespan_ab=timespan_ab, date_from=date_from)

trial.run_initial_dataload()
trial.feature_engineering(feature_set="scenarioa")

(1/3) Recv. and Stored GLW
(2/3) Recv. and Stored HIMX
(3/3) Recv. and Stored GLUU
Took 0:00:01.632216


 - GLW - 



 - HIMX - 



 - GLUU - 



In [107]:
X = trial.load_scenarioa_features("GLUU", True)
y = trial.load_scenarioa_labels("GLUU", True)

In [108]:
X.head()

,Date,Close,SMA_5,SMA_30,SMA_60,SMA_200,BOLL_5_UP,BOLL_5_DOWN,BOLL_30_UP,BOLL_30_DOWN,...,STOCOSCILATOR_14,STOCOSCILATOR_14_SMA,ADX,ADX_PDI,ADX_NDI,AROONUP_20,AROONDOWN_20,CHAIKIN_MFLOW_21,DAILY_MFLOW_21,OBV
0,2009-01-02,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.975366,-24.026597,-24.499908,NaN,NaN,NaN,-56200.000000,-56200.0
1,2009-01-05,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.975366,-24.026597,-24.499908,NaN,NaN,NaN,0.000000,-36000.0
2,2009-01-06,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.975366,-24.026597,-24.499908,NaN,NaN,NaN,180228.000000,159900.0
3,2009-01-07,0.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.975366,-24.026597,-24.499908,NaN,NaN,NaN,-65927.272727,56300.0
4,2009-01-08,0.70,0.63,NaN,NaN,NaN,0.881396,0.378604,NaN,NaN,...,NaN,NaN,-0.975366,-24.026597,-24.499908,NaN,NaN,NaN,-29100.000000,85400.0


In [109]:
y.head()

,Date,RETURN_1,RETURN_30,RETURN_60,RETURN_200
0,2009-01-02,0.020408,0.020408,-0.020408,1.632653
1,2009-01-05,0.520000,0.080000,0.060000,1.380000
2,2009-01-06,-0.078947,-0.250000,-0.197368,0.342105
3,2009-01-07,0.000000,-0.228571,-0.057143,0.571429
4,2009-01-08,0.157143,-0.257143,0.042857,0.642857


In [110]:
features_df = trial.load_scenarioa_features("GLUU", parseDate=True)
features_df.set_index("Date", inplace=True)

labels_df = trial.load_scenarioa_labels("GLUU", parseDate=True)
labels_df.set_index("Date", inplace=True)

model = scenarioa.create_model()
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace(features_df, labels_df, trial.train_from, trial.train_until, trial.test_from)
X_train


,Close,SMA_5,SMA_30,SMA_60,SMA_200,BOLL_5_UP,BOLL_5_DOWN,BOLL_30_UP,BOLL_30_DOWN,BOLL_60_UP,...,STOCOSCILATOR_14,STOCOSCILATOR_14_SMA,ADX,ADX_PDI,ADX_NDI,AROONUP_20,AROONDOWN_20,CHAIKIN_MFLOW_21,DAILY_MFLOW_21,OBV
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,1.21,1.144,1.085000,1.102333,0.98590,1.227187,1.060813,1.172059,0.997941,1.269153,...,90.909091,73.160173,-9.497432,-35.289645,-19.784891,100.0,40.0,-0.166518,6.344444e+04,5662300.0
2010-01-05,1.27,1.178,1.093667,1.102333,0.98975,1.300801,1.055199,1.199560,0.987774,1.269153,...,59.090909,72.222222,-11.599658,-23.031717,-22.536297,100.0,35.0,-0.167521,-7.502000e+04,5832800.0
2010-01-06,1.27,1.208,1.102000,1.102167,0.99360,1.333220,1.082780,1.222276,0.981724,1.268282,...,58.139535,69.379845,-12.142240,-19.595318,-26.474449,95.0,30.0,-0.235481,-1.166400e+05,6027200.0
2010-01-07,1.31,1.242,1.110000,1.102500,0.99760,1.367220,1.116780,1.251519,0.968481,1.270222,...,65.853659,61.028034,-13.558489,-19.994645,-25.941537,90.0,25.0,-0.207733,-1.114545e+04,6149800.0
2010-01-08,1.45,1.302,1.124000,1.105500,1.00235,1.482222,1.121778,1.309145,0.938855,1.290915,...,100.000000,74.664398,-15.627751,-19.961686,-20.429656,85.0,20.0,-0.118506,2.151000e+05,6364900.0
2010-01-11,1.43,1.346,1.136667,1.108167,1.00695,1.521271,1.170729,1.350617,0.922716,1.307300,...,83.720930,83.191530,-14.583870,-19.553711,-28.004654,100.0,15.0,-0.201292,-2.836000e+05,6081300.0
2010-01-12,1.38,1.368,1.148000,1.110333,1.01155,1.521883,1.214117,1.376301,0.919699,1.318374,...,71.428571,85.049834,-15.648564,-22.658894,-23.793813,95.0,10.0,-0.177265,3.166154e+04,5944100.0
2010-01-13,1.36,1.386,1.157000,1.112333,1.01615,1.497893,1.274107,1.396836,0.917164,1.327626,...,65.853659,73.667720,-13.225666,-29.585971,-29.448945,90.0,5.0,-0.166442,-1.596667e+04,5896200.0
2010-01-14,1.43,1.410,1.169667,1.114667,1.02090,1.486158,1.333842,1.425710,0.913623,1.340565,...,82.926829,73.403020,-16.549680,-17.379326,-28.911291,85.0,5.0,-0.133893,3.611111e+04,5961200.0


In [117]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True)

In [119]:
X_train

<class 'pandas.core.panel.Panel'>
Dimensions: 1510 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2010-01-04 00:00:00 to 2015-12-31 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [122]:
y_train.head()

,RETURN_1,RETURN_30,RETURN_60,RETURN_200
Date,,,,
2010-01-04,0.049587,-0.132231,-0.190083,0.363636
2010-01-05,0.000000,-0.181102,-0.212598,0.322835
2010-01-06,0.031496,-0.188976,-0.212598,0.393701
2010-01-07,0.106870,-0.259542,-0.206107,0.343511
2010-01-08,-0.013793,-0.365517,-0.275862,0.151724


In [123]:
X_test

<class 'pandas.core.panel.Panel'>
Dimensions: 327 (items) x 29 (major_axis) x 3 (minor_axis)
Items axis: 2016-01-04 00:00:00 to 2017-04-20 00:00:00
Major_axis axis: Close to OBV
Minor_axis axis: GLUU to HIMX

In [125]:
y_test.head()

,RETURN_1,RETURN_30,RETURN_60,RETURN_200
Date,,,,
2016-01-04,-0.012397,0.479339,0.165289,-0.148760
2016-01-05,-0.020921,0.564854,0.171548,-0.142259
2016-01-06,-0.064103,0.666667,0.153846,-0.136752
2016-01-07,0.100457,0.726027,0.365297,-0.082192
2016-01-08,0.020747,0.539419,0.224066,-0.149378


In [128]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True, "numpy")

In [132]:
X_train.shape

(1510, 29, 3)

In [135]:
y_train.shape

(1510, 4)

In [136]:
X_test.shape

(327, 29, 3)

In [137]:
y_test.shape

(327, 4)

In [150]:
model = scenarioa.create_model()

In [152]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 64, 27, 1)     640         convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 66, 29, 1)     0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 64, 27, 64)    640         zeropadding2d_2[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 110592)        0           convolution2d_9[0][0]            
___________________________________________________________________________________________

In [164]:
X_train, y_train, X_test, y_test = scenarioa.prepare_problemspace("GLUU", trial.valid_ticker_list(), trial.train_from, trial.train_until, trial.test_from, True, "pandas")

In [170]:
scenarioa.fit(model, X_train, y_train)

ValueError: Error when checking model input: expected convolution2d_input_7 to have 4 dimensions, but got array with shape (1510, 29, 3)